You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [2]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langgraph 0.0.53 requires langchain-core<0.3,>=0.2, but you have langchain-core 0.1.53 which is incompatible.
langgraph-checkpoint 2.0.10 requires langchain-core<0.4,>=0.2.38, but you have langchain-core 0.1.53 which is incompatible.

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
#%pip install crewai

from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [ ]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [6]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [8]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [15]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [16]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [17]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [18]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [19]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan on Artificial Intelligence

Title: The Future of Artificial Intelligence: Latest Trends and Key Players

Outline:

I. Introduction
- Define Artificial Intelligence (AI) and its significance in today's world
- Brief overview of the latest trends and advancements in AI
- Preview key players in the AI industry

II. Latest Trends in Artificial Intelligence
- Machine Learning and Deep Learning advancements
- AI in healthcare, finance, and other industries
- Ethical considerations and regulation in AI

- Display the results of your execution as markdown in the notebook.

In [21]:
from IPython.display import Markdown
Markdown(result)

# The Future of Artificial Intelligence: Latest Trends and Key Players

## Introduction

Artificial Intelligence (AI) has become an integral part of today's world, revolutionizing industries and changing the way we live and work. From self-driving cars to personalized recommendations on streaming platforms, AI is everywhere. The latest trends and advancements in AI continue to push the boundaries of what is possible, leading to exciting developments in machine learning, deep learning, and more. In this blog post, we will explore the key players in the AI industry and delve into the latest trends shaping the future of Artificial Intelligence.

## Latest Trends in Artificial Intelligence

### Machine Learning and Deep Learning Advancements
One of the most significant trends in AI is the continued advancements in machine learning and deep learning. These technologies are at the forefront of AI development, enabling computers to learn from data and improve their performance without being explicitly programmed. From image recognition to natural language processing, machine learning and deep learning are powering a wide range of applications across various industries.

### AI in Healthcare, Finance, and Other Industries
AI is making waves in healthcare, finance, and other industries, revolutionizing processes and improving outcomes. In healthcare, AI is being used to analyze medical images, diagnose diseases, and personalize treatment plans. In finance, AI is transforming fraud detection, risk assessment, and customer service. The potential of AI to drive innovation and efficiency in diverse sectors is limitless.

### Ethical Considerations and Regulation in AI Development
As AI technologies become more advanced, ethical considerations and regulation are becoming increasingly important. Issues such as bias in AI algorithms, data privacy, and the impact of AI on jobs are hot topics in the field. It is crucial for developers, policymakers, and society as a whole to address these concerns and ensure that AI is developed and used responsibly.

## Key Players in Artificial Intelligence

### Google DeepMind
Google DeepMind is a leading player in the AI industry, known for its groundbreaking work in deep reinforcement learning and AI research. The company's innovative approach to AI has led to major breakthroughs in areas such as healthcare, gaming, and robotics.

### IBM Watson
IBM Watson is another key player in AI, offering a wide range of AI-powered solutions for businesses and industries. From chatbots to predictive analytics, IBM Watson is helping organizations harness the power of AI to drive growth and innovation.

### Amazon Web Services AI
Amazon Web Services AI provides a comprehensive suite of AI services and tools for developers, making it easier to integrate AI into applications and workflows. With services such as Amazon SageMaker and Amazon Rekognition, AWS AI is empowering businesses to leverage AI for a competitive advantage.

### Microsoft Azure AI
Microsoft Azure AI is a cloud-based platform that offers a range of AI services and tools for developers and data scientists. From cognitive services to machine learning models, Microsoft Azure AI is enabling organizations to build and deploy AI solutions at scale.

### OpenAI
OpenAI is a research organization dedicated to advancing AI in a safe and beneficial way. Known for its work in natural language processing and reinforcement learning, OpenAI is shaping the future of AI through cutting-edge research and collaboration.

## Conclusion

As we look to the future of Artificial Intelligence, it is clear that the possibilities are endless. The latest trends and key players in the AI industry are driving innovation and shaping the way we interact with technology. By staying informed and engaged with the developments in AI, we can better understand its potential and impact on society. Subscribe to our newsletter for regular updates and insights on the latest trends and key players in Artificial Intelligence. Stay ahead of the curve and embrace the future of AI.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [22]:
topic = "Deepseek"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Deepseek.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Title: Exploring the Depths with Deepseek: A Comprehensive Guide

Introduction:
- Brief overview of Deepseek and its significance in the industry
- Mention of latest trends in Deepseek technology and applications
- Introduction to key players in the Deepseek market

Key Points:
1. Latest Trends in Deepseek:
- Advancements in underwater mapping technology
- Integration of artificial intelligence in Deepseek systems
- Growing popularity of Deepseek for marine research and exploration

2. Key Players in Deepseek:
- Deep Ocean

In [23]:
Markdown(result)

# Exploring the Depths with Deepseek: A Comprehensive Guide

## Introduction to Deepseek

The innovation of Deepseek technology has transformed the realm of ocean exploration by offering groundbreaking opportunities for marine research and underwater discovery. Through the integration of artificial intelligence and advancements in underwater mapping technology, Deepseek has emerged as an indispensable tool for scientists and researchers in the marine biology field. Leading companies in the Deepseek market, such as Deep Ocean Exploration Inc., Marine Robotics Ltd., and the Oceanographic Research Institute, are pioneering the development of cutting-edge technologies for deep-sea exploration.

## Latest Trends in Deepseek

The current trends in Deepseek technology are centered around enhancing underwater mapping capabilities and streamlining data collection processes for increased efficiency. Artificial intelligence plays a pivotal role in optimizing Deepseek systems, enabling more precise and detailed underwater surveys. As the evolution of Deepseek technology progresses, it is becoming increasingly popular for marine research and exploration, offering valuable insights into the enigmatic world of the deep sea.

## Key Players in the Deepseek Industry

Deep Ocean Exploration Inc., Marine Robotics Ltd., and the Oceanographic Research Institute stand as key figures in the Deepseek industry, driving innovation and pushing the boundaries of underwater exploration. These prominent players collaborate with research institutions and governmental bodies to unlock new discoveries and broaden our knowledge of the ocean. Through investments in cutting-edge technology and specialized expertise, they are shaping the future of deep-sea exploration.

## Noteworthy News and Developments

Recent breakthroughs achieved with Deepseek technology have captivated both the scientific community and the general public. Collaborative efforts between Deepseek companies and research institutions have resulted in groundbreaking discoveries and fresh insights into the mysteries of the ocean. With the continuous growth of the Deepseek industry, future prospects and developments promise exciting opportunities for exploration and discovery in the deep sea.

## Target Audience Analysis

The primary target audience for Deepseek technology includes researchers and scientists in the marine biology field, marine archaeologists, explorers, and technology enthusiasts with a keen interest in underwater exploration. By offering valuable insights and updates on the latest industry trends and developments, this blog post aims to engage and inform a diverse audience passionate about delving into the depths of the ocean.

## Addressing Audience Pain Points

Common pain points for the target audience include limited access to deep-sea exploration tools, lack of awareness regarding the latest Deepseek technologies, and challenges in finding reliable sources of information. This blog post seeks to alleviate these concerns by highlighting the advancements in Deepseek technology and showcasing the industry's key players, providing a valuable resource for readers intrigued by underwater exploration.

## Call to Action

Encouraging readers to explore the realm of Deepseek technology and its advancements, this blog post invites the audience to immerse themselves in the world of underwater exploration and uncover the marvels of the deep sea. By remaining informed and engaged with the latest developments in Deepseek technology, readers can actively participate in a community dedicated to unraveling the secrets of the ocean.

In conclusion, Deepseek technology presents a realm of endless possibilities for marine research, exploration, and revelation. By keeping abreast of the latest industry trends and developments, we can continue to push the boundaries of underwater exploration and unveil the mysteries concealed within the deep sea. Let us embark on this journey together and delve into the depths with Deepseek! 

*(Note: The viewpoints expressed in this blog post stem from the information outlined in the content plan and are intended to educate and captivate the reader)*

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).